In [8]:
#from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
from numpy.linalg import norm
import numpy as np 
import time
import math

In [9]:
# 计时装饰器
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        func(*args, **kwargs)
        end_time = time.time()
        print('Training time is :{:.2f} s.'.format(end_time - start_time))
    return wrapper

In [41]:
#两个list向量求和
def list_add(a,b):
    c = []
    for i in range(len(a)):
        c.append(a[i]+b[i])
    return c

In [59]:
class WaveletNeuralNet(object):
    # 初始化神经网络，sizes是神经网络的层数和每层神经元个数  
    def __init__(self, sizes):
        self.sizes_ = sizes
        self.num_layers_ = len(sizes)  # 层数
        if self.num_layers_ > 3:
            print('ERROR!')
        self.num_nuerals_ = sizes[1]#隐含层个数 w为
        self.w_ = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]  # w_、b_初始化为正态分布随机数
        self.b_ = [np.random.randn(y, 1) for y in sizes[1:]]#numpy.random.randn(d0, d1, …, dn)是从标准正态分布中返回一个或多个样本值
        self.t_ = np.random.randint(2, 15, (self.num_nuerals_, 1))#np.random.randint(low,high,size))
#         self.t_ = np.random.normal(5, 2., (self.num_nuerals_, 1))
        self.s_ = 2 * np.random.randn(self.num_nuerals_, 1)
        self.cell_=[np.random.randn(x) for x in sizes[1:len(sizes)-1]]
    
    #欧氏距离
    def oushi(self,v1,v2):
        v1=np.array(v1);v2=np.array(v2)
        op2=np.linalg.norm(v1-v2)
        return op2
    
    # Sigmoid函数，S型曲线，
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    # Sigmoid函数的导函数
    def sigmoid_der(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))
    
    # morlet小波母函数
    def phi(self, z, t=1, s=0):
        z_ = (z - s) / t
        return np.cos(1.75 * z_) * np.exp(-z_**2 / 2.)
    
    # 小波函数导数
    def phi_der(self, z, t=1, s=0):
        z_ = (z - s) / t
        return (-1.75 * np.sin(1.75 * z_) * np.exp(-z_**2 / 2) - z_ * np.cos(1.75 * z_) * np.exp(-z_**2 / 2)) / t
    
    def feedforward(self, x): # 前向：各层网络节点计算
        n = self.w_[0].shape[1]#输入层个数
        x = (np.array(x)).reshape(-1,1)#将数据准换成一行
        for i in range(self.num_layers_ -2):
            print(np.array(self.w_[i]).shape,np.array(self.b_[i]).shape)
            x1 = self.phi(np.dot(self.w_[i], list_add(x,self.cell_[i])) + self.b_[i], self.t_, self.s_)#np.dot点积 小波计算：输入层到输出层
            x=x1
            #print(x)
        x2 =np.dot(self.w_[-1], x1) + self.b_[-1]#self.sigmoid(np.dot(self.w_[1], x1) + self.b_[1])#sigmoid计算：
        '''隐含层到输出层'''
        return x2
    
    # 反向传播
    def backprop(self, x, y):#反向传播更新参数
        b_new = [np.ones(b.shape) for b in self.b_]
        w_new = [np.ones(w.shape) for w in self.w_]
        t_new = self.t_
        s_new = self.s_
        layer=self.num_layers_-1#权值向量的个数
        activation = x
        activations = [x]  # activations代表着每层的输出
        zs = []  # zs记录隐含层与输出层的数据
        for i in range(layer-1):#计算所有的z和a
            z = np.dot(self.w_[i],activation.reshape(-1,1))+ self.b_[i]#zl(1)
            #print('w0',np.array((self.w_[0])).shape)#print('z',np.array(z).shape)
            zs.append(z)#zl(1)
            activation = self.phi(list_add(np.array(z),np.array(self.cell_[i])), t_new, s_new)## 隐含层
            self.cell_[i]=z
            activations.append(activation)#记录了输入数据以及隐含层数据

        activation2=activations[-1]
        z = np.dot(self.w_[layer-1], activation2.reshape(-1,1)) + self.b_[layer-1]#zl(2)
        print('z',np.array(z).shape)
        zs.append(z) 
        activation = z#self.sigmoid(z)#sigmoid函数计算
        '''隐含层到输出层'''
        activations.append(activation)#至此记录了输入数据、隐含层数据以及输出数据
        
        #开始权值迭代计算
        delta = self.cost_derivative(activations[-1], y)#* zs[-1]#self.sigmoid_der(zs[-1])
        '''隐含层到输出层'''
        b_new[-1] = delta
        w_new[-1] = np.dot(delta, activations[-2].transpose())
        for i in range(2,layer+1,1):
            delta_last = delta.copy()
            z = zs[-i]
            sp = self.phi_der(z, t_new, s_new)
            delta = np.dot(self.w_[-i+1].transpose(), delta_last) * sp
            b_new[-i] =delta
            w_new[-i] = np.dot(delta, (activations[-(i+1)].reshape(-1,1)).transpose())
        sp_t = -.5 * t_new**-1.5 * self.phi((z-s_new) / t_new) - t_new**-2.5 * (z - s_new) * self.phi_der((z - s_new) / t_new)
        sp_s = -t_new**-1.5 * self.phi_der((z-s_new) / t_new)
        # t_new = np.dot(self.w_[-1].transpose(), delta_last)*sp_t # loss函数对小波函数缩放系数的偏导
        # s_new = np.dot(self.w_[-1].transpose(), delta_last)*sp_s # loss函数对小波函数平移系数的偏导
        
        t_new = delta * sp_t # loss函数对小波函数缩放系数的偏导
        s_new = delta * sp_s # loss函数对小波函数平移系数的偏导
        
        return (b_new, w_new, t_new, s_new)
        # 更新权值w，偏移b，缩放因子t，偏移因子s
    def update_mini_batch(self,data, mini_batch, lr):
        b_new = [np.zeros(b.shape) for b in self.b_]
        w_new = [np.zeros(w.shape) for w in self.w_]#a就是输入数据，b就是验证数据
        a, b = mini_batch,data
        n = np.float(mini_batch.size)
        x, y = np.array(a).reshape(-1, 1), np.array(b).reshape(-1, 1)
        delta_b_new, delta_w_new, t_new, s_new = self.backprop(x, y)
        #print('delta_b_new',(np.array(delta_b_new))[0].shape)
        #print('delta_w_new',(np.array(delta_w_new))[0].shape)
        b_new = [nb + dnb for nb, dnb in zip(b_new, delta_b_new)]
        w_new = [nw + dnw for nw, dnw in zip(w_new, delta_w_new)]
           
        self.w_ = [w - lr * nw for w, nw in zip(self.w_, w_new)]
        self.b_ = [b - lr * nb for b, nb in zip(self.b_, b_new)]
        self.t_ = self.t_ - lr * t_new
        self.s_ = self.s_ - lr * s_new
        #print('权值变化',self.w_[1][0]) #,np.array(self.w_[0]).shape,np.array(self.w_[1]).shape)
    
    # training_data是训练数据(x, y), epochs是训练次数, mini_batch_size是每次训练样本数, lr是learning rate，step是展示的迭代间隔
    @timer
    def SGD(self, training_data, epochs=50, mini_batch_size=50, lr=.1, step=10):
        assert type(step) == int, 'Step must be a integer.'
        n = training_data.shape[0]#数据个数
        juli=100
        i=0
        for j in range(epochs):
            ss = training_data
            i+=1
            print('第%d次循环'%(i))
            for k in range(0, n-mini_batch_size, mini_batch_size):
                j=1
                while juli>0.001 and j<1000:
                    np.random.shuffle(ss)#np.random.shuffle(x) 现场修改序列，改变自身内容。（类似洗牌，打乱顺序
                    mini_batches =ss[k:k + mini_batch_size]
                    #print('mini_batches' ,np.array(mini_batches).shape,mini_batches[0] )
                    cs0=ss[k+mini_batch_size:k+mini_batch_size+ 10] #此处对测试进行修改
                    #print('cs0' ,cs0 )
                    #print('cs0' ,np.array(cs0).shape,cs0[0] )
                    #print(np.array(cs0[0]).reshape(-1,1))
                    mini_batch=mini_batches
                    cs=cs0
                    self.update_mini_batch(cs, mini_batch, lr)
                    aa=self.feedforward(mini_batch)
                    print('输出结果',aa)
                    juli=self.oushi(aa,cs)
                    print('误差大小%d'%juli)
                    j+=1
                    print('内循环次数%d'%j)

# 计算正确率
    def evaluate(self, data):
        x_t, x_label = data
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in zip(list(x_t), list(x_label))]
        acc = sum(int(x == y) for (x, y) in test_results) / x_t.shape[0]
        return acc
    
    # mse_loss的导数
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
     # mse_loss
    def mse_loss(self, training_data):
        x_t,x_label = training_data
        test_results = [.5 * norm(self.feedforward(x).flatten() - self.one_hot(y, num_classes=10))**2
                        for (x, y) in zip(list(x_t), list(x_label))]
        return np.array(test_results).mean()
    
    # 预测
    def predict(self, data):
        data = data.reshape(-1, self.sizes_[0])
        value = np.array([np.argmax(net.feedforward(x)) for x in data], dtype='uint8')
        return value
    
    # 保存训练模型
    def save(self):
        pass  # 把_w和_b保存到文件(pickle)  
    
    def load(self):
        pas

In [ ]:
import pandas as pd
a0=pd.read_csv(r'C:\Users\stu\Desktop\xlab_时许数据\CPU.csv').values
a0[:,1].size;a1=a0[:,1];a1=a1.tolist();type(a1)
x=a1[:500]
x2=a1[500:510]
training_data=np.array(x)
net = WaveletNeuralNet([50,200,200,200,10])
net.SGD(training_data, epochs=2, mini_batch_size=50, lr=.1, step=20)

ERROR!
第1次循环
z (10, 1)
(200, 50) (200, 1)
(200, 200) (200, 1)
(200, 200) (200, 1)
输出结果 [[15.72664489]
 [14.28255284]
 [17.4855439 ]
 [16.84567495]
 [16.60632899]
 [16.71657324]
 [12.72629027]
 [12.9294277 ]
 [14.95079889]
 [19.13604371]]
误差大小423
内循环次数2
z (10, 1)
(200, 50) (200, 1)
(200, 200) (200, 1)
(200, 200) (200, 1)
输出结果 [[11.4326579 ]
 [12.56400774]
 [14.32791528]
 [12.94725327]
 [12.18881401]
 [13.38328742]
 [10.22136907]
 [12.58146711]
 [12.05218527]
 [12.49670932]]
误差大小457
内循环次数3
z (10, 1)
(200, 50) (200, 1)
(200, 200) (200, 1)
(200, 200) (200, 1)
输出结果 [[16.12841156]
 [17.37087001]
 [18.57651651]
 [17.97607076]
 [16.77167792]
 [17.42895446]
 [14.68190399]
 [16.49510446]
 [16.79161933]
 [17.17262661]]
误差大小410
内循环次数4
z (10, 1)
(200, 50) (200, 1)
(200, 200) (200, 1)
(200, 200) (200, 1)
输出结果 [[20.35416372]
 [21.51619296]
 [22.61185494]
 [22.04391036]
 [20.9257499 ]
 [21.55943919]
 [19.09638261]
 [20.69678878]
 [20.99271495]
 [21.36423961]]
误差大小367
内循环次数5
z (10, 1)
(200, 50) (200, 1

In [34]:
sizes=[150,210,220,230,10]
n=[np.random.randn(x) for x in sizes[1:len(sizes)-1]]
type(n)

list

In [54]:
a=[1,2,3,4]
b=[2,5,6]
c=[1,5,6]
new_list = map(lambda x,y: x + y, b,c)
#print((new_list))
type(new_list)
list_add(b,c)
print('the Sum is:%f' %(float(input("输入第一个数字："))+float(input("数入第二个数字：")))) 

输入第一个数字：1.5
数入第二个数字：5.6
the Sum is:7.100000


In [56]:
net.phi(1.5, t=1, s=0)

-0.28228765184873617

In [ ]:
class WaveletNeuralNet(object):
    # 初始化神经网络，sizes是神经网络的层数和每层神经元个数  
    def __init__(self, sizes):
        self.sizes_ = sizes
        self.num_layers_ = len(sizes)  # 层数
        if self.num_layers_ > 3:
            print('ERROR!')
        self.num_nuerals_ = sizes[1]#隐含层个数 w为
        self.w_ = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]  # w_、b_初始化为正态分布随机数
        self.b_ = [np.random.randn(y, 1) for y in sizes[1:]]#numpy.random.randn(d0, d1, …, dn)是从标准正态分布中返回一个或多个样本值
        self.t_ = np.random.randint(2, 15, (self.num_nuerals_, 1))#np.random.randint(low,high,size))
#         self.t_ = np.random.normal(5, 2., (self.num_nuerals_, 1))
        self.s_ = 2 * np.random.randn(self.num_nuerals_, 1)
    
    #欧氏距离
    def oushi(self,v1,v2):
        v1=np.array(v1);v2=np.array(v2)
        op2=np.linalg.norm(v1-v2)
        return op2
    
    # Sigmoid函数，S型曲线，
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    # Sigmoid函数的导函数
    def sigmoid_der(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))
    
    # morlet小波母函数
    def phi(self, z, t=1, s=0):
        z_ = (z - s) / t
        return np.cos(1.75 * z_) * np.exp(-z_**2 / 2.)
    
    # 小波函数导数
    def phi_der(self, z, t=1, s=0):
        z_ = (z - s) / t
        return (-1.75 * np.sin(1.75 * z_) * np.exp(-z_**2 / 2) - z_ * np.cos(1.75 * z_) * np.exp(-z_**2 / 2)) / t
    
    def feedforward(self, x): # 前向：各层网络节点计算
        n = self.w_[0].shape[1]#输入层个数
        x = (np.array(x)).reshape(-1,1)#将数据准换成一行
        for i in range(self.num_layers_ -2):
            print(np.array(self.w_[i]).shape,np.array(self.b_[i]).shape)
            x1 = self.phi(np.dot(self.w_[i], x) + self.b_[i], self.t_, self.s_)#np.dot点积 小波计算：输入层到输出层
            x=x1
            #print(x)
        x2 =np.dot(self.w_[-1], x1) + self.b_[-1]#self.sigmoid(np.dot(self.w_[1], x1) + self.b_[1])#sigmoid计算：
        '''隐含层到输出层'''
        return x2
    
    # 反向传播
    def backprop(self, x, y):#反向传播更新参数
        b_new = [np.ones(b.shape) for b in self.b_]
        w_new = [np.ones(w.shape) for w in self.w_]
        t_new = self.t_
        s_new = self.s_
        layer=self.num_layers_-1#权值向量的个数
        activation = x
        activations = [x]  # activations代表着每层的输出
        zs = []  # zs记录隐含层与输出层的数据
        for i in range(layer-1):#计算所有的z和a
            z = np.dot(self.w_[i],activation.reshape(-1,1))+ self.b_[i]#zl(1)
            #print('w0',np.array((self.w_[0])).shape)#print('z',np.array(z).shape)
            zs.append(z)#zl(1)
            activation = self.phi(z, t_new, s_new)## 隐含层
            activations.append(activation)#记录了输入数据以及隐含层数据

        activation2=activations[-1]
        z = np.dot(self.w_[layer-1], activation2.reshape(-1,1)) + self.b_[layer-1]#zl(2)
        print('z',np.array(z).shape)
        zs.append(z) 
        activation = z#self.sigmoid(z)#sigmoid函数计算
        '''隐含层到输出层'''
        activations.append(activation)#至此记录了输入数据、隐含层数据以及输出数据
        
        #开始权值迭代计算
        delta = self.cost_derivative(activations[-1], y)#* zs[-1]#self.sigmoid_der(zs[-1])
        '''隐含层到输出层'''
        b_new[-1] = delta
        w_new[-1] = np.dot(delta, activations[-2].transpose())
        for i in range(2,layer+1,1):
            delta_last = delta.copy()
            z = zs[-i]
            sp = self.phi_der(z, t_new, s_new)
            delta = np.dot(self.w_[-i+1].transpose(), delta_last) * sp
            b_new[-i] =delta
            w_new[-i] = np.dot(delta, (activations[-(i+1)].reshape(-1,1)).transpose())
        sp_t = -.5 * t_new**-1.5 * self.phi((z-s_new) / t_new) - t_new**-2.5 * (z - s_new) * self.phi_der((z - s_new) / t_new)
        sp_s = -t_new**-1.5 * self.phi_der((z-s_new) / t_new)
        # t_new = np.dot(self.w_[-1].transpose(), delta_last)*sp_t # loss函数对小波函数缩放系数的偏导
        # s_new = np.dot(self.w_[-1].transpose(), delta_last)*sp_s # loss函数对小波函数平移系数的偏导
        
        t_new = delta * sp_t # loss函数对小波函数缩放系数的偏导
        s_new = delta * sp_s # loss函数对小波函数平移系数的偏导
        
        return (b_new, w_new, t_new, s_new)
        # 更新权值w，偏移b，缩放因子t，偏移因子s
    def update_mini_batch(self,data, mini_batch, lr):
        b_new = [np.zeros(b.shape) for b in self.b_]
        w_new = [np.zeros(w.shape) for w in self.w_]#a就是输入数据，b就是验证数据
        a, b = mini_batch,data
        n = np.float(mini_batch.size)
        x, y = np.array(a).reshape(-1, 1), np.array(b).reshape(-1, 1)
        delta_b_new, delta_w_new, t_new, s_new = self.backprop(x, y)
        #print('delta_b_new',(np.array(delta_b_new))[0].shape)
        #print('delta_w_new',(np.array(delta_w_new))[0].shape)
        b_new = [nb + dnb for nb, dnb in zip(b_new, delta_b_new)]
        w_new = [nw + dnw for nw, dnw in zip(w_new, delta_w_new)]
           
        self.w_ = [w - lr * nw for w, nw in zip(self.w_, w_new)]
        self.b_ = [b - lr * nb for b, nb in zip(self.b_, b_new)]
        self.t_ = self.t_ - lr * t_new
        self.s_ = self.s_ - lr * s_new
        #print('权值变化',self.w_[1][0]) #,np.array(self.w_[0]).shape,np.array(self.w_[1]).shape)
    
    # training_data是训练数据(x, y), epochs是训练次数, mini_batch_size是每次训练样本数, lr是learning rate，step是展示的迭代间隔
    @timer
    def SGD(self, training_data, epochs=50, mini_batch_size=50, lr=.1, step=10):
        assert type(step) == int, 'Step must be a integer.'
        n = training_data.shape[0]#数据个数
        juli=100
        i=0
        for j in range(epochs):
            ss = training_data
            i+=1
            print('第%d次循环'%(i))
            for k in range(0, n-mini_batch_size, mini_batch_size):
                j=1
                while juli>0.001 and j<1000:
                    np.random.shuffle(ss)#np.random.shuffle(x) 现场修改序列，改变自身内容。（类似洗牌，打乱顺序
                    mini_batches =ss[k:k + mini_batch_size]
                    #print('mini_batches' ,np.array(mini_batches).shape,mini_batches[0] )
                    cs0=ss[k+mini_batch_size:k+mini_batch_size+ 10] #此处对测试进行修改
                    #print('cs0' ,cs0 )
                    #print('cs0' ,np.array(cs0).shape,cs0[0] )
                    #print(np.array(cs0[0]).reshape(-1,1))
                    mini_batch=mini_batches
                    cs=cs0
                    self.update_mini_batch(cs, mini_batch, lr)
                    aa=self.feedforward(mini_batch)
                    print('输出结果',aa)
                    juli=self.oushi(aa,cs)
                    print('误差大小%d'%juli)
                    j+=1
                    print('内循环次数%d'%j)
        
        #print('第%d次循环',%i)
                
            '''
            print(self.oushi(aa,cs))
            d1=self.oushi(aa,cs)
            accur = self.evaluate(training_data) * 100
            mse_loss = self.mse_loss(training_data)
            if (j + 1) % step == 0 or j == 0:
                print("Epoch {0}, mse_loss: {1:.4f}, accury on the training set :{2:.2f}{3}".format(j+1, mse_loss, accur, '%'))
            # print("Epoch {0}: {1} / {2}".format(j, self.evaluate(training_data), n))
            '''
# 计算正确率
    def evaluate(self, data):
        x_t, x_label = data
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in zip(list(x_t), list(x_label))]
        acc = sum(int(x == y) for (x, y) in test_results) / x_t.shape[0]
        return acc
    
    # mse_loss的导数
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
     # mse_loss
    def mse_loss(self, training_data):
        x_t,x_label = training_data
        test_results = [.5 * norm(self.feedforward(x).flatten() - self.one_hot(y, num_classes=10))**2
                        for (x, y) in zip(list(x_t), list(x_label))]
        return np.array(test_results).mean()
    
    # 预测
    def predict(self, data):
        data = data.reshape(-1, self.sizes_[0])
        value = np.array([np.argmax(net.feedforward(x)) for x in data], dtype='uint8')
        return value
    
    # 保存训练模型
    def save(self):
        pass  # 把_w和_b保存到文件(pickle)  
    
    def load(self):
        pas